In [1]:
import sys
sys.path.append("../")  # add parent directory to the system path
from utils_data import *
from utils_model import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from time import time

2023-09-30 20:42:12.158999: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-30 20:42:12.771101: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# get the data ready
root_name = "Research"
train_path, test_path, val_path = make_data_path(root_name=root_name)

DataGenerator = ImageDataGenerator(rescale=1.0 / 255)
train_data, test_data, val_data = make_data_ready(
    DataGenerator, train_path, test_path, val_path
)

Found 84479 images belonging to 524 classes.
Found 2620 images belonging to 524 classes.
Found 2620 images belonging to 524 classes.


In [3]:
# ________________ CONFIG THE BASE MODELS ________________ #
weights = "imagenet"
input_shape = (224, 224, 3)
num_class = 524

include_top = False
base_trainable = False
pooling = "max"
learning_rate = 0.1
epochs = 1


model_funcs = [
    tf.keras.applications.inception_v3.InceptionV3,
    tf.keras.applications.resnet50.ResNet50,
    tf.keras.applications.mobilenet.MobileNet,
    tf.keras.applications.mobilenet_v2.MobileNetV2,
    tf.keras.applications.efficientnet.EfficientNetB0,
    tf.keras.applications.efficientnet_v2.EfficientNetV2B0,
]

model_0s = {}

for model_func in model_funcs:
    model_name = model_func.__name__

    model_0s[model_name] = configure_model_base(
        model_func=model_func,
        weights=weights,
        include_top=False,
        base_trainable=False,
        input_shape=input_shape,
        pooling=pooling,
        num_class=num_class,
        learning_rate=learning_rate,
    )

2023-09-30 20:42:20.458394: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-30 20:42:20.475691: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-30 20:42:20.476100: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-30 20:42:20.477303: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-30 20:42:20.477651: I tensorflow/compile

In [4]:
# ________________ TEST TRAIN THE BASE MODELS ________________ #
history = {}
metrics = {}

for model_name, model_0 in model_0s.items():
    # train the models

    start_time = time()

    history[model_name] = model_0.fit(
        train_data,
        epochs=epochs,
        steps_per_epoch=len(train_data),
        validation_data=val_data,
        validation_steps=int(0.25 * len(val_data)),
    )

    end_time = time()
    execution_time = end_time - start_time

    # save metrics for the models
    metrics[model_name] = {
        "train_loss": history[model_name].history["loss"],
        "train_accuracy": history[model_name].history["accuracy"],
        "val_loss": history[model_name].history["val_loss"],
        "val_accuracy": history[model_name].history["val_accuracy"],
        "execution_time": execution_time,
    }

    # evaluate the model on test data and save the results
    test_loss, test_accuracy = model_0.evaluate(test_data)
    metrics[model_name]["test_loss"] = test_loss
    metrics[model_name]["test_accuracy"] = test_accuracy

2023-09-30 20:42:39.328389: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-09-30 20:42:41.590938: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-30 20:42:43.673649: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x319edcc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-30 20:42:43.673697: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2023-09-30 20:42:43.746374: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


   1/2640 [..............................] - ETA: 7:38:01 - loss: 9.9543 - accuracy: 0.0000e+00

2023-09-30 20:42:44.076952: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-30 20:42:44.158966: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


82/82 [==============================] - 18s 223ms/step - loss: 212.5779 - accuracy: 0.6668


2023-09-30 21:12:33.476666: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_4/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


82/82 [==============================] - 15s 178ms/step - loss: 1989.2185 - accuracy: 0.0019


2023-09-30 21:23:48.979878: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_5/efficientnetv2-b0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


82/82 [==============================] - 10s 123ms/step - loss: 261.0547 - accuracy: 0.0019


In [5]:
file_path = "../output/data"
file_name = "model_performance_log.json"
save_metric(metrics, file_name, file_path)

In [6]:
file_path = "../output/data"
file_name = "model_performance_mobilenet_finetune2.json"

loaded_metrics = load_dict_from_json(file_name, file_path)

nrows = 6
ncols = 2
width = 10
height = nrows * width/ncols

figure = visualize_metric(loaded_metrics, nrows=nrows, ncols=ncols, figsize=(width, height))
# save_plot(figure, file_name, file_path)

CustomException: Error occured in the script, name: [/home/hoangng/Projects/Bird_classification/Research/notebook/../utils_model.py], line number: [126] error message: [[Errno 2] No such file or directory: '../output/data/model_performance_mobilenet_finetune2.json']